In [15]:
import pandas as pd
from forex_python.converter import CurrencyRates
from datetime import datetime
c = CurrencyRates()
# get rates for all unique dates, added tqdm progress bar to see progress
from tqdm import tqdm  
rates_dict = pd.DataFrame({date: CurrencyRates().get_rates('EUR', date_obj=pd.to_datetime(date, unit='D'))
              for date in tqdm(df['date_order'].unique())})
# now use these rates to set cost to 1/(RUB to currency_x rate), except when currency is RUB and when servicecost is 0, in those cases just use servicecost
#df['XR'] = df.apply(lambda row: 1.0/rates_dict[row['date_order']][row['rate']]*row['ci=ost'] if row['rate']!='RUB' and row['cost'] != 0 else row['cost'], axis=1)
rates_rub = rates_dict.T
rates_rub = rates_rub.loc[:,['RUB']]
rates_rub.index.name = 'date_order'

100%|██████████| 12/12 [00:05<00:00,  2.27it/s]


In [17]:
df = df.merge(rates_rub,on='date_order')
df.rename(columns = {'RUB': 'XR'}, inplace = True)
df.loc[(df['Country'] =='Russian Federation'),'XR'] = 1
df['cost_RUB'] = df.cost * df.XR

In [29]:
# Calculate supply (total per mill) = done
# Converted all deliveries at historic rate (in RUB) = done
# Check and remove outliers per destination
import seaborn as sns
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set()
# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

Fetch the distance between points (latitude, longitude) and calculate distance 

In [30]:
import geocoder
from pprint import pprint
key = '484ae73bfa5b43d28bde740af59c0c0c' # get api key from: https://opencagedata.com
mills = dict.fromkeys(demand['точка отгрузки'])
cities = dict.fromkeys(demand['город'])
for name in mills:
    result = geocoder.opencage(name, key=key)
    mills[name] = result.latlng
for name in cities:
    result = geocoder.opencage(name, key=key)
    cities[name] = result.latlng

In [31]:
#Transform, rename, merge
mills_distance = pd.DataFrame.from_dict(mills, orient='index',columns=['lat_точка','long_точка'])
cities_distance = pd.DataFrame.from_dict(cities, orient='index',columns=['lat_город','long_город'])
mills_distance.index.name = 'точка отгрузки'
cities_distance.index.name = 'город'
demand = demand.merge(mills_distance, left_on='точка отгрузки', right_on='точка отгрузки')
demand = demand.merge(cities_distance, left_on='город', right_on='город')

In [32]:
#Calculate distance between two points, calculate cost per km = proxy for cost minimization function
import h3
demand['расстояние'] = demand.apply(lambda row: h3.point_dist((row['lat_точка'], row['long_точка']), (row['lat_город'], row['long_город'])), axis=1)
demand['стоимость за километр'] = demand.предложение_RUB/demand.расстояние

In [33]:
#issue with inf type of float, which json doesn't recognise
demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']] = demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']].astype(str)
demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']] = demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']].astype(str)